# Bandpass filter

In [ ]:
import os
import nibabel as nib
import numpy as np
from scipy import signal


def apply_bandpass_filter(file_path, output_path):

    fmri_scan = nib.load(file_path)

    # Define cutoff frequencies relative to the Nyquist frequency
    TR = fmri_scan.header['pixdim'][4]
    sampling_frequency = 1 / TR
    nyquist_frequency = 0.5 * sampling_frequency
    low_cutoff = 0.01 / nyquist_frequency
    high_cutoff = 0.08 / nyquist_frequency

    # Digital bandpass filter
    filter_order = 4
    b, a = signal.butter(filter_order, [low_cutoff, high_cutoff], btype='band', analog=False)

    # Apply the filter to each voxel's time series
    filtered_fmri_data = np.zeros_like(fmri_scan.get_fdata())

    for i in range(filtered_fmri_data.shape[0]):
        for j in range(filtered_fmri_data.shape[1]):
            for k in range(filtered_fmri_data.shape[2]):
                voxel_time_series = fmri_scan.get_fdata()[i, j, k, :]
                filtered_voxel_time_series = signal.lfilter(b, a, voxel_time_series)
                filtered_fmri_data[i, j, k, :] = filtered_voxel_time_series

    filtered_img = nib.Nifti1Image(filtered_fmri_data, affine=fmri_scan.affine)
    nib.save(filtered_img, output_path)

general_directory = 'E:/Internship/Scans/CN' # This directory changes depending on the group analyzed

output_directory = 'E:/Internship/Scans/CN/PostBandpass' # It also changes depending on the group

# Create if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Initialize a counter for creating subdirectories
sub_counter = 1

# Function to recursively search for files starting with 'swu' and ending with '.nii', output shape of SPM 12 for functional preprocessing 
def search_and_filter(directory):
    global sub_counter
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.startswith('swu') and filename.endswith('.nii'):
                input_file_path = os.path.join(root, filename)
                
                # Create a subdirectory for each processed file, this makes the data handling easier on GIFT's group ICA
                sub_directory = os.path.join(output_directory, f'sub{sub_counter}')
                os.makedirs(sub_directory, exist_ok=True)
                
                output_file_path = os.path.join(sub_directory, f'filtered_{filename}')
                apply_bandpass_filter(input_file_path, output_file_path)
                
                sub_counter += 1

search_and_filter(general_directory)

print("Filtering complete.")

